In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('usecase_7').getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

In [4]:
uber_df = spark.read.format('csv').options(header=True, inferSchema=True).load('/FileStore/tables/uber')

In [5]:
uber_df.count()

Out[5]: 354

In [6]:
uber_df.printSchema()

root
-- dispatching_base_number: string (nullable = true)
-- date: string (nullable = true)
-- active_vehicles: integer (nullable = true)
-- trips: integer (nullable = true)

In [7]:
uber_df.show(5)

+-----------------------+--------+---------------+-----+
dispatching_base_number| date|active_vehicles|trips|
+-----------------------+--------+---------------+-----+
 B02512|1/1/2015| 190| 1132|
 B02765|1/1/2015| 225| 1765|
 B02764|1/1/2015| 3427|29421|
 B02682|1/1/2015| 945| 7679|
 B02617|1/1/2015| 1228| 9537|
+-----------------------+--------+---------------+-----+
only showing top 5 rows

In [8]:
uber_df.createOrReplaceTempView('uber')

In [9]:
from pyspark.sql.functions import desc,dayofweek,month,year
from pyspark.sql.types import StringType
from datetime import datetime

In [10]:
def udfToDate(input):
    dateString, rStr = "", ""
    format1 = "%M/%d/%Y"
    format2 = "%Y-%M-%d"
    try:
        if (input.strip().lower() == "null" or not(bool(input.strip()))):
            print('Inside if')
            rStr = dateString
        else:
            print('Inside else')
            rStr = datetime.strptime(input, format1).strftime(format2)
    except Exception as e:
        print(e)
    return rStr


In [11]:
def getDay(input):
    rstr = ""
    days = {1:'Sun', 2:'Mon', 3:'Tue', 4:'Wed', 5:'Thu', 6:'Fri', 7:'Sat'}
    try:
      rstr = days[input]
    except Exception as e:
      print(e)
    return rstr

In [12]:
spark.udf.register('udfToDate',udfToDate, StringType())
spark.udf.register('getDay',getDay, StringType())

Out[86]: <function __main__.getDay(input)>

In [13]:
spark.sql('SELECT dayofweek(udfToDate(date)) as day_of_week from uber group by day_of_week').show()

+-----------+
day_of_week|
+-----------+
 1|
 6|
 3|
 5|
 4|
 7|
 2|
+-----------+

In [14]:
uber_df1 = spark.sql("select getDay(dayofweek(udfToDate(date))) as valid_day, *  from uber")

In [15]:
uber_df1.show(5)

+---------+-----------------------+--------+---------------+-----+
valid_day|dispatching_base_number| date|active_vehicles|trips|
+---------+-----------------------+--------+---------------+-----+
 Thu| B02512|1/1/2015| 190| 1132|
 Thu| B02765|1/1/2015| 225| 1765|
 Thu| B02764|1/1/2015| 3427|29421|
 Thu| B02682|1/1/2015| 945| 7679|
 Thu| B02617|1/1/2015| 1228| 9537|
+---------+-----------------------+--------+---------------+-----+
only showing top 5 rows

In [16]:
uber_df1.groupBy('dispatching_base_number','valid_day').sum().select('sum(trips)','dispatching_base_number','valid_day').orderBy(desc('sum(trips)')).show(6)

+----------+-----------------------+---------+
sum(trips)|dispatching_base_number|valid_day|
+----------+-----------------------+---------+
 356789| B02764| Sat|
 326968| B02764| Fri|
 304200| B02764| Thu|
 249896| B02764| Sun|
 241137| B02764| Wed|
 221343| B02764| Tue|
+----------+-----------------------+---------+
only showing top 6 rows

In [17]:
spark.stop()